In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import sys
import hopsworks
from config import hopsworks_api_key
import great_expectations as ge
from config import Airport_Coordinates, Arlanda

In [7]:
def fetch_smhi_weather(latitude, longitude):
    url = f"https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/{longitude}/lat/{latitude}/data.json"

    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()

        forecasts = []

        for ts in data['timeSeries'[:24]]:
            valid_time = datetime.fromisoformat(ts['validTime'].replace('Z', '+00:00'))
            params = {p['name']: p['values'][0] for p in ts['parameters']}

            forecast = {
                'timestamp': valid_time.strftime('%Y-%m-%d %H:%M:%S'),
                'temperature': params.get('t', None), # In Celsius
                'wind_speed': params.get('ws', None), # In m/s
                'wind_direction': params.get('wd', None), # In Degrees
                'humidity': params.get('r', None), # In %
                'pressure': params.get('msl', None), # In hPa
                'precipitation': params.get('pcat', None), # Category
                'visibility': params.get('vis', None), # In km
                'cloud_cover': params.get('tcc_mean', None) # In Octas/Oktas
            }

            forecasts.append(forecast)

        return forecasts

    except Exception as e:
        print(f"Error fetching weather forecast at Arlanda: {e}")
        return []

In [8]:
def create_weather_condition(precipitation, visibility, wind_speed):
    if precipitation in [1,2]:
        return 'snow'
    elif precipitation == 3:
        if wind_speed > 10:
            return 'rain_windy'
        else:
            return 'rain'
    elif visibility and visibility < 1:
        return 'fog'
    elif wind_speed and wind_speed > 15:
        return 'windy'
    else:
        return 'clear'

In [23]:
def fetch_arlanda_weather():
    arlanda_data = []

    coords = Airport_Coordinates[Arlanda]
    forecasts = fetch_smhi_weather(coords['lat'], coords['long'])

    for f in forecasts:
        f['airport_code'] = Arlanda
        f['weather_condition'] = create_weather_condition(f.get('precipitation'), f.get('visibility'), f.get('wind_speed'))
        f['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        arlanda_data.append(f)

    return pd.DataFrame(arlanda_data)

In [24]:
df_weather = fetch_arlanda_weather()

## Uploading Created Dataframe on Hopsworks

In [16]:
project = hopsworks.login(api_key_value=hopsworks_api_key, host="eu-west.cloud.hopsworks.ai")

2025-12-30 17:36:01,247 INFO: Initializing external client
2025-12-30 17:36:01,248 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-30 17:36:02,877 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3207


In [25]:
fs = project.get_feature_store()

In [26]:
weather_features_fg = fs.get_or_create_feature_group(
    name='weather_features',
    description="Weather Features at Arlanda Airport",
    version=1,
    primary_key = ["airport_code", "timestamp"],
)

In [27]:
weather_features_fg.insert(df_weather)

Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/fs/3151/fg/3266


Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████████████████████████████| Rows 77/77 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/jobs/named/weather_features_1_offline_fg_materialization/executions


(Job('weather_features_1_offline_fg_materialization', 'SPARK'), None)

In [28]:
weather_features_fg.update_feature_description("temperature", "In Celsius")
weather_features_fg.update_feature_description("wind_speed", "In metres/second")
weather_features_fg.update_feature_description("wind_direction", "In Degrees")
weather_features_fg.update_feature_description("humidity", "In Percentages (%)")
weather_features_fg.update_feature_description("pressure", "In hPa (HectoPascals)")
weather_features_fg.update_feature_description("precipitation", "Categories where 1-Snow, 2-Mixed Snow, 3-Rain, 4-Drizzle")
weather_features_fg.update_feature_description("visibility", "In kilometres")
weather_features_fg.update_feature_description("cloud_cover", "In Octas/Oktas")